# Inequality of underrepresnted groups in PyData leadership

### Or what's up at the top of the PyData ecosystem?

So, I am the host of [Open Source Directions](https://www.quansight.com/open-source-directions), the webinar series (and *yes* soon podcast) about the roadmaps of projects in the PyData / Scientific Python space. I would like for the series to be is a welcoming venue for the less-often-heard voices of the PyData community. Unfortunately, by focusing on projects and their lead/core developers, I (and other Project leads) believe that we are reinforcing existing biases and overrepresentation.

Yes, there are steps we can take to address the diveristy issues on Open Source Directions. We have adjusted our processes and procedures, with more significant changes to come in the future. I personally welcome any and all feedback in this regard. Feel free to reach out to me publicly or privately with your ideas and concerns.

But this post is not about that. This post is to show that we in the NumFOCUS and PyData community have vast inequities in representation in the leadership of our projects.


## How should equity be measured?

One of my big pet peeves is that people usually discuss diversity & equality as a ratio between men/women. This is a terrible way to talk about this problem for a couple reasons:

1. There are many other axes of inequality, including economic class, social class, racial, ethnic, religious,
   educational, and more!
2. Ratios only work for binary systems, and gender is not binary.
3. Equality & inequality should be bounded from $[0, 1]$, i.e. they should be fractional measures of how   
   equitable a system is. A men/women ratio is instead bounded from $[0, \infty]$, where both $0$ and $\infty$ 
   are perfectly unequal, and only $1$ is perfectly equal.

I understand why people use men/women ratios. They are deceptively easy to understand. They probably track with other systemic inequities. In terms of population differences, it is (again deceptively) close to pairity with only [0.3-0.5% of the U.S. population identifying as transgender](https://www.nytimes.com/2015/06/09/upshot/the-search-for-the-best-estimate-of-the-transgender-population.html). That doesn't make such ratios any less wrong.

Additionally, it is very difficult to obtain reasonable data on inequality along avenues other than gender. This makes geneder the easiest metric to obtain.

In my [PyData Carolinas Keynote](https://youtu.be/fFKztg_ZRB4) ([slides](https://docs.google.com/presentation/d/148_jbdBCC4WjGmhY8s1wo8G3Yn2dZ1bc-PDBeg_Kp-w/edit?usp=sharing)) from a few years ago, I presented (what I feel is) a much better, information-theoritic, entropy-based model of equality & inequality. For a 3-gendered partitioning scheme (female, male, nonbinary), the Generalized Entropy Inequality measure (GEI, $G$) reduces to, 

$G = \ln(3) - H$

Where $H$ is our friend the Shannon entropy, or

$H = -\sum_{i=1}^S p_i \ln p_i$

$G$ has much better mathematical properties than a simple ratio.

## Methodology

I have gone through the NumFOCUS fiscally sponsored projects and tried to determine gender of their leadership teams in a couple of ways:

1. First, I looked at their websites.  If they listed as stearing comittee (as with Jupyter) or a core team (as with conda-forge), I would use this as the representative body.
2. If no leadership team was clearly posted, I would go to the repository and look at their top contributors. Yes, this subjects the project to the "tyranny of code contrbution," but I tried to avoid that in (1)! I defined top contributors as those with more than 150 commits.
3. I am trying to be as generous as possible, including all past members and subcommittee members (such as in pandas).

If you find problems with my counting, please put in a PR to this repo that updates the `data.json` file!  I welcome all fixes here.

## Results 

## Active vs. Passive Diversity Problems

In discussing these issues with [Chris "CJ" Wright](https://billingegroup.github.io/people/cwright.html), the president of Columbia University's [qSTEM](https://www.facebook.com/QSTEMColumbia/) group (their LGBTQ+ STEM organization), and a close friend, there is a difference between *active* and *passive* diversity issues, where these terms are defined as:

* **Active:** There is a toxic member of a community - or a toxic culture as a whole - which prevents an
  equitable system of representation from evolving or destroys existing equitable systems.
* **Passive:** There are significant biaseses, in which the agents in the system will be unable to make major 
  strides toward equality by simply continuing the existing day-to-day activities.
  
I believe (anecdotally) that PyData has passive diversity issues with respect to project leadership. Over the years, we have made a ton of progress towards equitability (thanks to Gina Helfrich and other members of [DISC](https://numfocus.org/programs/diversity-inclusion)) on issues such as conference attendees, conference speakers, keynotes, board members, etc. However, this has not translated down to project leadership.